In [1]:
import os
import json
import urllib.request
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException



import re

In [2]:
driver_path = r"F:\Fork_git\Labelling_Menu_Data\menu_scraper\webdriver\chromedriver-win64\chromedriver.exe"


In [3]:
service = Service(executable_path=driver_path)

options = Options()
# options.add_argument("--headless")  # Enable headless mode
# options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(70, 91)}.0.4472.124 Safari/537.36')

driver = webdriver.Chrome(service=service)



In [4]:
def close_popups_if_any(driver):
    # If there's a known popup or overlay (by some selector), try to close it.
    try:
        # Assuming the popup has a close button with identifiable attribute or text, e.g., a class 'close-btn'
        close_button = driver.find_element(By.CSS_SELECTOR, ".close-btn")  # Adjust this selector
        close_button.click()
    except NoSuchElementException:
        pass  # No such element, likely no popup

def interact_with_element_with_retry(driver, element, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            element.click()  # Or any other interaction
            return  # Successful interaction
        except (ElementNotInteractableException, StaleElementReferenceException) as e:
            print(f"Retrying interaction with element: {e}")
            attempt += 1
            time.sleep(2)  # Brief pause before retrying
    print("Failed to interact with the element after several attempts.")

In [5]:

# Set of URLs to visit, initialized with the starting page
urls_to_crawl = set(["https://malasichuan.co.uk/"])  # Replace with your starting URL
crawled_urls = set()

docx_pattern = re.compile(r".*\.docx")
pdf_pattern = re.compile(r".*\.pdf")
image_pattern = re.compile(r".*menu.*\.(jpg|jpeg|png)")

# Regular expression to identify PDF files (you might need to adjust this)
link_keywords = ['pdf', 'menu', 'dish']  # Add more relevant keywords as needed

In [6]:



def find_and_click_elements():
    try:
        # Wait for the page to load completely
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*')))

        # Search for elements that contain the text 'menu' (case-insensitive) and might be clickable
        elements = driver.find_elements(By.XPATH, "//*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'menu')]")
        
        for elem in elements:
            try:
                # Scroll the element into view
                actions = ActionChains(driver)
                actions.move_to_element(elem).perform()

                # Use JavaScript to click the element in case it's being overlapped by something else
                driver.execute_script("arguments[0].click();", elem)
                
                print(f"Clicked on element with text: {elem.text}")
                time.sleep(2)  # Pause to allow any new content to load
                
            except StaleElementReferenceException:
                # In case the page content changes dynamically and reference to the element is lost
                print("Skipped stale element")
                continue
            except ElementClickInterceptedException:
                # Handle the case where another element is blocking this one
                print("Element was not clickable at the moment. Skipping.")
                continue
            except ElementNotInteractableException:
                # Handle the case where the element is not interactable
                print("Element not interactable, skipping.")
                continue

    except TimeoutException:
        # Handle a timeout situation when the page takes too long to load completely
        print("Timed out waiting for page to load")


def find_menus():
    # Gather all the links, frames, and images on the page
    
    links = driver.find_elements(By.TAG_NAME, "a")
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    images = driver.find_elements(By.TAG_NAME, "img")
    
    # Check each link to see if it's text or href attribute indicates it could be a menu
    for link in links:
        
        try:
            link_text = link.text.lower()
            href = link.get_attribute('href')
            if link_text: 
                if 'menu' in link_text:
                    print(f"Possible menu link found: {href} with link_text {link_text}")
            if href:
                if docx_pattern.match(href) or pdf_pattern.match(href):
                    print(f"Possible menu link found: {href}")
                
        except NoSuchElementException:
            continue

    # Check frames' src attributes for document links
    for frame in frames:
        try:
            src = frame.get_attribute('src')
            if src:
                if docx_pattern.match(src) or pdf_pattern.match(src):
                    print(f"Possible menu document in frame: {src}")
                
        except NoSuchElementException:
            continue

    # Check images' src or alt attributes for indications they could be menus
    for img in images:
        try:
            src = img.get_attribute('src')
            alt = img.get_attribute('alt').lower()
            if alt:
                if 'menu' in alt:
                    print(f"Possible menu image found: {alt}")

            if src:
                if image_pattern.match(src):
                    print(f"Possible menu image found: {src}")
                
        except NoSuchElementException:
            continue

def crawl(url):
    print(crawled_urls)
    driver.get(url)
    crawled_urls.add(url)

    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*')))  # Waiting for visibility
    except TimeoutException:
        print("Timed out waiting for page to load")
        return

    close_popups_if_any(driver)

    find_and_click_elements()  # Function to click elements revealing more content
    find_menus()

    links = driver.find_elements(By.TAG_NAME, "a")
    # find_menu_pdf(links)

    for link in links:
        href = link.get_attribute('href')

        # Avoid crawling the same page or outside websites
        if href in crawled_urls or href is None or not href.startswith(driver.current_url):
            continue

        urls_to_crawl.add(href)





In [7]:
# Crawl through the queue until it's empty
while urls_to_crawl:
    current_url = urls_to_crawl.pop()
    print(f"Crawling: {current_url}")
    crawl(current_url)
    time.sleep(2) 

Crawling: https://malasichuan.co.uk/
set()
Element not interactable, skipping.
Element not interactable, skipping.
Skipped stale element
Skipped stale element
Possible menu link found: https://malasichuan.co.uk/menu with link_text menu
Possible menu link found: http://malasichuan.co.uk/wp-content/uploads/2023/07/Ma-La-Menu_07-2023.pdf
Possible menu link found: http://malasichuan.co.uk/wp-content/uploads/2022/09/Ma-La-Drink-Menu-08-22.pdf
Possible menu link found: http://malasichuan.co.uk/wp-content/uploads/2022/08/Ma-La-Dessert-Menu-Aug-2022.pdf
Crawling: https://malasichuan.co.uk/menu
{'https://malasichuan.co.uk/'}
Element not interactable, skipping.
Element not interactable, skipping.
Element not interactable, skipping.
Skipped stale element
Skipped stale element
Skipped stale element
Skipped stale element
Skipped stale element
Possible menu link found: https://malasichuan.co.uk/menu with link_text menu
Possible menu link found: http://malasichuan.co.uk/wp-content/uploads/2023/07/Ma-

In [ ]:
# # Close the browser once done
# driver.quit()